# Delays for departure airports

In [ ]:
%load_ext sql
from flights import settings
%sql $settings.DB_URL

In [ ]:
%%sql
select departure_airport,
       avg(actual_departure - scheduled_departure) as avg_delay
from flight
group by departure_airport;